In [1]:
import os
import shutil
import sqlite3
import xml.etree.ElementTree as ET
import re
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import datetime
import shutil
import tkinter as tk
from tkinter import filedialog

def select_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    folder_path = filedialog.askdirectory(initialdir=r"H:\Pitching\Data", title="Select Folder")
    if folder_path:
        copy_and_move_files(folder_path)


def extract_name(folder_name):
    # Extract the full name from the folder name
    # Assuming the folder name is in the format "Last, First"
    parts = folder_name.split(", ")
    if len(parts) == 2:
        first_name = parts[1].strip()
        last_name = parts[0].strip()
        full_name = f"{first_name} {last_name}"
    else:
        full_name = folder_name.strip()
    return full_name


def copy_and_move_files(base_path):
    # Get the base folder name
    base_folder = os.path.basename(base_path)

    # Get list of folders in the base path
    folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    # Sort folders by creation time
    folders.sort(key=lambda x: os.path.getctime(os.path.join(base_path, x)), reverse=True)

    if not folders:
        print("No folders found in the specified directory.")
        return

    latest_folder = folders[0]
    folder_path = os.path.join(base_path, latest_folder)

    # Get creation time of the latest folder
    creation_time = os.path.getctime(folder_path)
    creation_date = datetime.datetime.fromtimestamp(creation_time).strftime('%m-%d-%y')

    # Find the files to copy and rename
    files_to_copy_rename = []
    for file in os.listdir(folder_path):
        if file.lower() == 'session.xml' or file.lower() == 'session_data.xml':
            files_to_copy_rename.append(file)

    if not files_to_copy_rename:
        print("No XML files named 'session.xml' or 'session_data.xml' found in the latest folder.")
        return

    # Construct new file names
    new_file_names = []
    initials = extract_name(base_folder)
    for file in files_to_copy_rename:
        if file.lower() == 'session.xml':
            new_file_name = f"session_{initials}_{creation_date}.xml"
        elif file.lower() == 'session_data.xml':
            new_file_name = f"session_data_{initials}_{creation_date}.xml"
        else:
            continue
        new_file_names.append(new_file_name)

    # Create temporary directory to store renamed copies
    temp_dir = os.path.join(base_path, "temp_rename")
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)

    # Copy and rename the files to the temporary directory
    for original_file, new_file_name in zip(files_to_copy_rename, new_file_names):
        original_file_path = os.path.join(folder_path, original_file)
        new_file_path = os.path.join(temp_dir, new_file_name)
        shutil.copyfile(original_file_path, new_file_path)
        print(f"File '{original_file}' copied and renamed to: {new_file_path}")

    # Copy the renamed files to the destination folder
    new_folder_path = r"G:\My Drive\Pitching Data\New Data"
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)
    for new_file_name in new_file_names:
        temp_file_path = os.path.join(temp_dir, new_file_name)
        destination_file_path = os.path.join(new_folder_path, new_file_name)
        shutil.copyfile(temp_file_path, destination_file_path)
        print(f"File '{new_file_name}' copied to: {destination_file_path}")

    # Clean up temporary directory
    shutil.rmtree(temp_dir)


# Prompt user to select the folder
select_folder()


# Source and destination directory paths
source_dir = r"G:\My Drive\Pitching Data\New Data"
destination_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

# Iterate over files in the source directory
for file_name in os.listdir(source_dir):
    if file_name.endswith('.xml'):
        source_file_path = os.path.join(source_dir, file_name)
        destination_file_path = os.path.join(destination_dir, file_name)
        shutil.move(source_file_path, destination_file_path)
        print(f"File '{file_name}' moved to: {destination_file_path}")

print("XML files copied successfully!")

# This code is *chef's kiss* perfect. Does everything for parsing for variables, iterates over it
# multiple times, and includes the specific filename for each trial. I love it.


# Connect to the SQLite database
conn = sqlite3.connect("Cover_Page.db")
cursor = conn.cursor()

# Drop the table if it exists
cursor.execute('DROP TABLE IF EXISTS variables')

# Create the variables table
cursor.execute('''
   CREATE TABLE variables (
       id INTEGER PRIMARY KEY AUTOINCREMENT,
       session_data_id TEXT NOT NULL,
       file_name TEXT NOT NULL,
       Pitch TEXT,
       Score REAL,
       Linear_Pelvis_Speed REAL,
       HSS_Footplant REAL,
       Pelvis_Ang_Footplant REAL,
       Trunk_Ang_Footplant REAL,
       Pelvic_Obl REAL,
       Front_Leg_Brace REAL,
       Front_Leg_Var_Val REAL,
       Lead_Leg_Midpoint REAL,
       Lead_Leg_GRF_y REAL,
       Lead_Leg_GRF_z REAL,
       Lead_Leg_GRF_x REAL,
       Horizontal_Abduction REAL,
       Shld_ER_Footplant REAL,
       Shld_ER_Max REAL,
       Lateral_Trunk_Tilt REAL,
       Pelvis_Ang_Velo REAL,
       Torso_Ang_Velo REAL,
       Arm_Ang_Velo REAL,
       MPH REAL,
       Trunk_Ang_MER REAL,
       Trunk_Ang_Rel REAL,
       Pelvis_Ang_MER REAL,
       Pelvis_Ang_Rel REAL,
       HSS_MER REAL,
       HSS_Rel REAL,
       Front_Leg_Footplant REAL,
       Front_Leg_MER REAL,
       Front_Leg_Rel REAL,
       Abduction_Max REAL,
       Hand_Ang_Velo REAL,
       Stride_Length REAL,
       Arm_Slot REAL,
       Weight REAL
   )
''')

# Commit the changes before proceeding to parsing XML files
conn.commit()

# List of XML files to parse (replace this with your actual directory path)
directory_path = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

while True:
    # Get a list of XML files in the specified directory
    xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

    # Track the number of new data entries
    new_data_entries = 0

    for file_name in xml_files:
        try:
            # Check if the file has already been processed
            cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
            count = cursor.fetchone()[0]

            if count == 0:
                # Parse the XML file
                tree = ET.parse(file_name)
                root = tree.getroot()

                # Extract session_data_id from the file name
                session_data_id_match = re.search(r"session_data_(.+)\.xml", file_name)
                if session_data_id_match:
                    session_data_id = session_data_id_match.group(1)
                else:
                    session_data_id = None

                # Check if session_data_id is None, and skip the file if it is
                if session_data_id is not None:
                    # Iterate over all occurrences of "owner" element with a value containing "Fastball"
                    for owner_element in root.findall('.//owner'):
                        if 'Fastball' in owner_element.get('value', ''):
                            fastball_owner = owner_element
                            # Extract and insert data into the database
                            linear_pelvis_speed = None
                            hss_footplant = None
                            pelvis_ang_fp = None
                            trunk_ang_fp = None
                            pelvis_obl = None
                            front_leg_brace = None
                            front_leg_var_val = None
                            lead_leg_midpoint = None
                            lead_leg_grf_y = None
                            lead_leg_grf_z = None
                            lead_leg_grf_x = None
                            horizontal_abduction = None
                            shld_er_fp = None
                            shld_er_max = None
                            lateral_trunk_tilt = None
                            pelvis_ang_velo = None
                            torso_ang_velo = None
                            arm_ang_velo = None
                            Trunk_Ang_MER = None
                            Trunk_Ang_Rel = None
                            Pelvis_Ang_MER = None
                            Pelvis_Ang_Rel = None
                            HSS_MER = None
                            HSS_Rel = None
                            lead_knee_ang_at_footstrike = None
                            Front_Leg_MER = None
                            lead_knee_ang_at_release = None
                            Abduction_Max = None
                            Hand_Ang_Velo = None
                            Stride_Length = None
                            Arm_Slot = None
                            weight = weight

                            # Extract the value of "owner"
                            pitch_value = fastball_owner.get('value', '')

                            for variable_element in fastball_owner.findall('.//name[@value]'):
                                variable_name = variable_element.attrib['value']
                                component_x_element = variable_element.find('./component[@value="X"]')
                                component_y_element = variable_element.find('./component[@value="Y"]')
                                component_z_element = variable_element.find('./component[@value="Z"]')

                                if variable_name == "MaxPelvisLinearVel_MPH" and component_y_element is not None:
                                    linear_pelvis_speed = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Hip Shoulders Sep@Footstrike" and component_z_element is not None:
                                    hss_footplant = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Shoulder_Angle@Release" and component_x_element is not None:
                                    Arm_Slot = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Hip Shoulders Sep@Max_Shoulder_Rot" and component_z_element is not None:
                                    HSS_MER = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Hip Shoulders Sep@Release" and component_z_element is not None:
                                    HSS_Rel = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Angle@Footstrike" and component_z_element is not None:
                                    pelvis_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Angle@Max_Shoulder_Rot" and component_z_element is not None:
                                    Pelvis_Ang_MER = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Angle@Release" and component_z_element is not None:
                                    Pelvis_Ang_Rel = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Trunk_Angle@Footstrike" and component_z_element is not None:
                                    trunk_ang_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Trunk_Angle@Max_Shoulder_Rot" and component_z_element is not None:
                                    Trunk_Ang_MER = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Trunk_Angle@Release" and component_z_element is not None:
                                    Trunk_Ang_Rel = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Footstrike" and component_x_element is not None:
                                    lead_knee_ang_at_footstrike = float(
                                        component_x_element.attrib['data'].replace(',', '.'))
                                    lead_knee_ang_at_release = float(root.find(
                                        './/name[@value="Lead_Knee_Angle@Release"]/component[@value="X"]').attrib[
                                                                         'data'].replace(',', '.'))
                                    front_leg_brace = lead_knee_ang_at_footstrike - lead_knee_ang_at_release
                                if variable_name == "Lead_Knee_Angle@Footstrike" and component_y_element is not None:
                                    lead_knee_var_val_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                if variable_name == "Lead_Knee_Angle@Max_Shoulder_Rot" and component_x_element is not None:
                                    Front_Leg_MER = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Knee_Angle@Release" and component_y_element is not None:
                                    lead_knee_var_val_rel = float(component_y_element.attrib['data'].replace(',', '.'))
                                    # Calculate the difference and store in front_leg_var_val
                                    front_leg_var_val = lead_knee_var_val_fp - lead_knee_var_val_rel
                                if variable_name == "Pelvis_Angle@Footstrike" and component_y_element is not None:
                                    pelvis_obl_fp = float(component_y_element.attrib['data'].replace(',', '.'))
                                    pelvis_angle_release_element = root.find(
                                        './/name[@value="Pelvis_Angle@Release"]/component[@value="Y"]')
                                    if pelvis_angle_release_element is not None:
                                        pelvis_obl_release = float(
                                            pelvis_angle_release_element.attrib['data'].replace(',', '.'))
                                        pelvis_obl = pelvis_obl_release - pelvis_obl_fp
                                elif variable_name == "Lead_Leg_GRF_mag_Midpoint_FS_Release" and component_x_element is not None:
                                    lead_leg_midpoint = abs(float(component_x_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Lead_Leg_GRF_min" and component_y_element is not None:
                                    lead_leg_grf_y = abs(float(component_y_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Lead_Leg_GRF_max" and component_z_element is not None:
                                    lead_leg_grf_z = float(component_z_element.attrib['data'].replace(',', '.'))
                                if variable_name == "Lead_Leg_GRF_max" and component_x_element is not None:
                                    lat_force_max = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Lead_Leg_GRF_min" and component_x_element is not None:
                                    lat_force_min = float(component_x_element.attrib['data'].replace(',', '.'))
                                    # Calculate the sum of absolute values and store in lead_leg_grf_x
                                    lead_leg_grf_x = abs(lat_force_max) + abs(lat_force_min)
                                elif variable_name == "Pitching_Shoulder_Angle@Footstrike" and component_x_element is not None:
                                    horizontal_abduction = abs(
                                        float(component_x_element.attrib['data'].replace(',', '.')))
                                if variable_name == "Pitching_Shoulder_Angle@Footstrike":
                                    if component_z_element is not None:
                                        shld_er_fp = float(component_z_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Shoulder_Angle_Max" and component_z_element is not None:
                                    shld_er_max = abs(float(component_z_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Pitching_Shoulder_Angle_Min" and component_x_element is not None:
                                    Abduction_Max = abs(float(component_x_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "Trunk_wrt_Pelvis_FE@Release" and component_y_element is not None:
                                    lateral_trunk_tilt = float(component_y_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pelvis_Ang_Vel_max" and component_x_element is not None:
                                    pelvis_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Thorax_Ang_Vel_max" and component_x_element is not None:
                                    torso_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Hand_Ang_Vel_max" and component_x_element is not None:
                                    Hand_Ang_Velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Humerus_Ang_Vel_max" and component_x_element is not None:
                                    arm_ang_velo = float(component_x_element.attrib['data'].replace(',', '.'))
                                elif variable_name == "Pitching_Shoulder_Angle_Max" and component_z_element is not None:
                                    shld_er_max = abs(float(component_z_element.attrib['data'].replace(',', '.')))
                                elif variable_name == "STRIDE_LENGTH" and component_x_element is not None:
                                    Stride_Length = abs(float(component_x_element.attrib['data'].replace(',', '.')))
                                

                                linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
                                hss_footplant = hss_footplant if hss_footplant is not None else 30
                                pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
                                trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
                                pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
                                front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
                                front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
                                lead_leg_midpoint = lead_leg_midpoint if lead_leg_midpoint is not None else 1.5
                                lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
                                lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
                                lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
                                horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
                                shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
                                lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
                                shld_er_max = shld_er_max if shld_er_max is not None else 160
                                pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
                                torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
                                arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000

                                # Insert the data into the database
                            cursor.execute('''
                               INSERT INTO variables (
                                   session_data_id,
                                   file_name,
                                   Pitch,
                                   Linear_Pelvis_Speed,
                                   HSS_Footplant,
                                   Pelvis_Ang_Footplant,
                                   Trunk_Ang_Footplant,
                                   Pelvic_Obl,
                                   Front_Leg_Brace,
                                   Front_Leg_Var_Val,
                                   Lead_Leg_Midpoint,
                                   Lead_Leg_GRF_y,
                                   Lead_Leg_GRF_z,
                                   Lead_Leg_GRF_x,
                                   Horizontal_Abduction,
                                   Shld_ER_Footplant,
                                   Shld_ER_Max,
                                   Lateral_Trunk_Tilt,
                                   Pelvis_Ang_Velo,
                                   Torso_Ang_Velo,
                                   Arm_Ang_Velo,
                                   MPH,
                                   Trunk_Ang_MER,
                                   Trunk_Ang_Rel,
                                   Pelvis_Ang_MER,
                                   Pelvis_Ang_Rel,
                                   HSS_MER,
                                   HSS_Rel,
                                   Front_Leg_Footplant,
                                   Front_Leg_MER,
                                   Front_Leg_Rel,
                                   Abduction_Max,
                                   Hand_Ang_Velo,
                                   Stride_Length,
                                   Arm_Slot,
                                   Weight
                               ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                           ''', (
                            session_data_id, file_name, pitch_value, linear_pelvis_speed, hss_footplant, pelvis_ang_fp,
                            trunk_ang_fp, pelvis_obl, front_leg_brace, front_leg_var_val, lead_leg_midpoint, lead_leg_grf_y,
                            lead_leg_grf_z, lead_leg_grf_x, horizontal_abduction, shld_er_fp, shld_er_max,                            lateral_trunk_tilt, pelvis_ang_velo, torso_ang_velo, arm_ang_velo, None, Trunk_Ang_MER, Trunk_Ang_Rel, Pelvis_Ang_MER, Pelvis_Ang_Rel, HSS_MER, HSS_Rel, lead_knee_ang_at_footstrike, Front_Leg_MER, lead_knee_ang_at_release,Abduction_Max, Hand_Ang_Velo, Stride_Length, Arm_Slot, weight))

                            # Increment the count of new data entries
                            new_data_entries += 1
                else:
                    print(f"Skipping file {file_name} due to missing session_data_id.")

        except Exception as e:
            # Print the error and the file name where it occurred
            print(f"Error processing file {file_name}: {e}")

    # Commit the changes for each file
    conn.commit()

    # If no new data entries, break out of the loop
    if new_data_entries == 0:
        break

# Close the connection
conn.close()

print("Data inserted into the database.")

# Connect to the database
conn = sqlite3.connect("Cover_Page.db")
cursor = conn.cursor()

directory_path = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"

# Get a list of XML files in the specified directory
xml_files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".xml")]

# Track the number of new data entries
new_data_entries = 0

# Parse the XML files
for file_name in xml_files:
    try:
        # Check if the file has already been processed
        cursor.execute('SELECT COUNT(*) FROM variables WHERE file_name = ?', (file_name,))
        count = cursor.fetchone()[0]

        if count == 0:
            # Parse the XML file
            tree = ET.parse(file_name)
            root = tree.getroot()

            # Extract session_data_id from the file name
            session_data_id_match = re.search(r"session_(.+)\.xml", file_name)
            if session_data_id_match:
                session_data_id = session_data_id_match.group(1)
            else:
                session_data_id = None

            # Check if session_data_id is None, and skip the file if it is
            if session_data_id is not None:
                # Extract Weight
                weight_element = root.find('./Fields/Weight')
                weight = float(weight_element.text) if weight_element is not None else None

                # Iterate over all occurrences of "Measurement" element
                for measurement_element in root.findall('.//Measurement'):
                    pitch_value = measurement_element.get('Filename')
                    used_element = measurement_element.find('./Fields/Used')

                    if pitch_value is not None and used_element is not None:
                        # Replace "qtm" with "c3d" in pitch_value
                        pitch_value = pitch_value.replace("qtm", "c3d")

                        # Print statement for debugging
                        print(f"Filename: {pitch_value}, Used: {used_element.text}")

                        if 'Fastball' in pitch_value and used_element.text == 'True':
                            # Extract the values of "Comments" (MPH)
                            comments_value_element = measurement_element.find('./Fields/Comments')

                            mph_value = None
                            if comments_value_element is not None:
                                mph_value = float(comments_value_element.text)
                                print(f"MPH: {mph_value}, Weight: {weight}, Session ID: {session_data_id}")

                            # Update the MPH and Weight values in the "variables" table
                            cursor.execute('''
                                UPDATE variables
                                SET MPH = ?, Weight = ?
                                WHERE session_data_id = ? AND Pitch = ?
                            ''', (mph_value, weight, session_data_id, pitch_value))

                            # Increment the new data entries counter
                            new_data_entries += 1
                    else:
                        print("Filename or Used element not found.")

    except Exception as e:
        print(f"Error processing file {file_name}: {e}")

# Commit the changes
conn.commit()

cursor.execute('SELECT id, Weight, Lead_Leg_Midpoint, Lead_Leg_GRF_y, Lead_Leg_GRF_z, Lead_Leg_GRF_x FROM variables')
rows = cursor.fetchall()

for row in rows:
    id, weight, midpoint, grf_y, grf_z, grf_x = row

    # Set Weight to 195 if it is None
    if weight is None or weight <= 0:
        weight = 88

    # Calculate the adjustment factor based on Weight
    adjustment_factor = weight * 9.81

    # Adjust values if they are above 4
    if midpoint and midpoint > 4:
        midpoint /= adjustment_factor
    if grf_y and grf_y > 4:
        grf_y /= adjustment_factor
    if grf_z and grf_z > 4:
        grf_z /= adjustment_factor
    if grf_x and grf_x > 4:
        grf_x /= adjustment_factor

    # Update the database with the adjusted values
    cursor.execute('''
        UPDATE variables
        SET Lead_Leg_Midpoint = ?, Lead_Leg_GRF_y = ?, Lead_Leg_GRF_z = ?, Lead_Leg_GRF_x = ?, Weight = ?
        WHERE id = ?
    ''', (midpoint, grf_y, grf_z, grf_x, weight, id))

# Commit the adjusted values
conn.commit()

print("Ground reaction forces adjusted based on Weight.")

# Close the connection
conn.close()

print(f"Number of new data entries: {new_data_entries}")
print("Data inserted into the database.")


# Move xml files from New Data folder to appropriate folder
# Define source directory and destination directories
source_dir = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\New Data"
dest_dir_data = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\Kinematic Data\Pro"
dest_dir_mph = r"C:\Users\q\PycharmProjects\Rename_Session.xml\Pitching Data\MPH Data\Pro"

# Ensure the destination directories exist
os.makedirs(dest_dir_data, exist_ok=True)
os.makedirs(dest_dir_mph, exist_ok=True)

# Iterate through files in the source directory
for file_name in os.listdir(source_dir):
    source_file_path = os.path.join(source_dir, file_name)
    
    # Check if it's a file (not a directory)
    if os.path.isfile(source_file_path):
        if "session_data_" in file_name:
            # Move file containing "session_data_" to the kinematic data folder
            dest_path = os.path.join(dest_dir_data, file_name)
            shutil.move(source_file_path, dest_path)
            print(f"Moved {file_name} to {dest_dir_data}")
        elif "session_" in file_name:
            # Move file containing "session_" to the MPH data folder
            dest_path = os.path.join(dest_dir_mph, file_name)
            shutil.move(source_file_path, dest_path)
            print(f"Moved {file_name} to {dest_dir_mph}")
            
            
# Generates Score
#
# Connect to the SQLite database
conn = sqlite3.connect("Cover_Page.db")
cursor = conn.cursor()

# Select all rows from the 'variables' table
cursor.execute('SELECT * FROM variables')
rows = cursor.fetchall()

# Iterate through each row and calculate the score
for row in rows:
    linear_pelvis_speed = row[5]
    hss_footplant = row[6]
    pelvis_ang_fp = row[7]
    trunk_ang_fp = row[8]
    pelvis_obl = row[9]
    front_leg_brace = row[10]
    front_leg_var_val = row[11]
    lead_leg_midpoint = row[12]
    lead_leg_grf_y = row[13]
    lead_leg_grf_z = row[14]
    lead_leg_grf_x = row[15]
    horizontal_abduction = row[16]
    shld_er_fp = row[17]
    shld_er_max = row[18]
    lateral_trunk_tilt = row[19]
    pelvis_ang_velo = row[20]
    torso_ang_velo = row[21]
    arm_ang_velo = row[22]
    MPH = row[23]

    # Check for None values and replace with 0
    linear_pelvis_speed = linear_pelvis_speed if linear_pelvis_speed is not None else 4
    hss_footplant = hss_footplant if hss_footplant is not None else 30
    pelvis_ang_fp = pelvis_ang_fp if pelvis_ang_fp is not None else 60
    trunk_ang_fp = trunk_ang_fp if trunk_ang_fp is not None else 90
    pelvis_obl = pelvis_obl if pelvis_obl is not None else 2
    front_leg_brace = front_leg_brace if front_leg_brace is not None else 8
    front_leg_var_val = front_leg_var_val if front_leg_var_val is not None else 0
    lead_leg_midpoint = lead_leg_midpoint if lead_leg_midpoint is not None else 1.5
    lead_leg_grf_y = lead_leg_grf_y if lead_leg_grf_y is not None else .7
    lead_leg_grf_z = lead_leg_grf_z if lead_leg_grf_z is not None else 1.5
    lead_leg_grf_x = lead_leg_grf_x if lead_leg_grf_x is not None else .25
    horizontal_abduction = horizontal_abduction if horizontal_abduction is not None else 20
    shld_er_fp = shld_er_fp if shld_er_fp is not None else 45
    lateral_trunk_tilt = lateral_trunk_tilt if lateral_trunk_tilt is not None else 35
    shld_er_max = shld_er_max if shld_er_max is not None else 160
    pelvis_ang_velo = pelvis_ang_velo if pelvis_ang_velo is not None else 600
    torso_ang_velo = torso_ang_velo if torso_ang_velo is not None else 1000
    arm_ang_velo = arm_ang_velo if arm_ang_velo is not None else 5000
    MPH = MPH if MPH is not None else 75

    # Calculate the score using the specified equation
    score = (0.05 * linear_pelvis_speed) + (0.15 * front_leg_brace) + \
            (5 * lead_leg_midpoint) + \
            (0.55 * horizontal_abduction) + (0.012 * torso_ang_velo) + (.05 * pelvis_obl) + (
                        .6 * trunk_ang_fp) - (.45 * pelvis_ang_fp) + (.1 * shld_er_max) + (.1 * front_leg_var_val) + (
                        .05 * pelvis_ang_velo) + MPH

    # Update the 'Score' column in the database
    cursor.execute('UPDATE variables SET Score = ? WHERE id = ?', (score, row[0]))

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Scores updated in the database.")


XML files copied successfully!
Data inserted into the database.
Ground reaction forces adjusted based on Weight.
Number of new data entries: 0
Data inserted into the database.
Scores updated in the database.


In [2]:
# Create Dash Board

import dash
from dash import html, dcc
import sqlite3
from scipy.stats import percentileofscore
import numpy as np

app = dash.Dash(__name__)

# Database paths
db_path = r"C:/Users/q/PycharmProjects/Rename_Session.xml/Cover_Page.db"
db_reference_path = r"C:/Users/q/PycharmProjects/Rename_Session.xml/grading_equation_reference_data_Pro.db"

# List of columns to pull and calculate percentiles for
columns_to_pull = [
    "session_data_id", "Front_Leg_Brace", "HSS_Footplant", "Pelvis_Ang_Footplant", "Trunk_Ang_Footplant",
    "Horizontal_Abduction", "Shld_ER_Footplant", "Shld_ER_Max", "Pelvis_Ang_Velo", "Torso_Ang_Velo",
    "Arm_Ang_Velo", "MPH", "Trunk_Ang_MER", "Trunk_Ang_Rel", "Pelvis_Ang_MER", "Pelvis_Ang_Rel", 
    "Score", "HSS_MER", "HSS_Rel", "Front_Leg_Footplant", "Front_Leg_MER", "Front_Leg_Rel", 
    "Abduction_Max", "Hand_Ang_Velo"
]

# Fetch the row with the highest score
def fetch_highest_score_row(db_path, columns):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    column_query = ", ".join(columns)
    query = f"SELECT {column_query} FROM variables ORDER BY Score DESC LIMIT 1;"
    cursor.execute(query)
    result = cursor.fetchone()
    conn.close()

    if result:
        return {columns[i]: result[i] for i in range(len(columns))}
    return {}

def calculate_percentiles(db_reference_path, data_row, columns):
    conn_ref = sqlite3.connect(db_reference_path)
    cursor_ref = conn_ref.cursor()
    percentiles = {}

    for column in columns:
        # Skip if the column does not exist or the value is None
        if column not in data_row or data_row[column] is None:
            percentiles[column] = None
            continue

        try:
            # Ensure the current value is numeric
            current_value = float(data_row[column])
        except ValueError:
            percentiles[column] = None
            continue

        cursor_ref.execute(f"SELECT CAST({column} AS REAL) FROM variables WHERE {column} IS NOT NULL")
        reference_values = [row[0] for row in cursor_ref.fetchall() if row[0] is not None]

        # Convert reference_values to a numeric array
        reference_values = np.array(reference_values, dtype=np.float64)

        if reference_values.size > 0:
            percentiles[column] = percentileofscore(reference_values, current_value, kind='mean')
        else:
            percentiles[column] = None

    conn_ref.close()
    return percentiles


# Fetch data and calculate percentiles
data_row = fetch_highest_score_row(db_path, columns_to_pull)
percentile_values = calculate_percentiles(db_reference_path, data_row, columns_to_pull)

# Merge percentile data into data_row
for key, value in percentile_values.items():
    data_row[f"{key}_percentile"] = value

# Helper function to create a table
def create_table(rows, cols, col_widths, data):
    header_row = html.Tr([
        html.Th(col, style={'width': f"{col_width}px", 'height': '40px', 'fontWeight': 'normal', 'fontSize': '12px', 'padding': '4px'}) for col, col_width in zip(cols, col_widths)
    ])

    body_rows = [
        html.Tr([
            html.Td(data[i][j], style={
                'width': f"{col_widths[j]}px", 'height': '36px', 'padding': '4px',
                'textAlign': 'right' if j == 0 else 'center'
            }) for j in range(len(cols))
        ]) for i in range(rows)
    ]

    return html.Table([
        html.Thead(header_row),
        html.Tbody(body_rows)
    ])

def extract_name_and_date(session_data_id):
    if session_data_id:
        parts = session_data_id.split("_")  # Split by underscore
        name = parts[0]  # Extract name
        raw_date = parts[1] if len(parts) > 1 else "Unknown Date"  # Extract date
        # Reformat date to MM/DD/YY
        formatted_date = raw_date.replace("-", "/")
        return name, formatted_date
    return "", "Unknown Date"

# Extract Name and Date from `session_data_id`
if data_row and "session_data_id" in data_row:
    name, date = extract_name_and_date(data_row["session_data_id"])
    data_row["Name"] = name
    data_row["Date"] = date
else:
    data_row["Name"] = "Unknown Player"
    data_row["Date"] = "Unknown Date"

# Layout
app.layout = html.Div([
    # Background layer
    html.Div(style={'backgroundColor': '#dfe0e0', 'width': '100vw', 'height': '100vh', 'position': 'absolute', 'zIndex': '-2'}),
    
    # Background image layer
    html.Div(
        style={
            'position': 'absolute',
            'width': '1878.5px',
            'height': '120%',
            'left': '35.9px',
            'top': '173.5px',
            'backgroundImage': 'url("https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png")',
            'backgroundSize': 'contain',
            'backgroundRepeat': 'no-repeat',
            'opacity': '0.15',
            'zIndex': '-1'
        }
    ),
    
    # Player's Name
    html.Div(data_row.get("Name", ""), style={
        'position': 'absolute',
        'left': '50%',
        'top': '40px',
        'transform': 'translateX(-50%)',
        'fontSize': '56px',
        'fontFamily': 'Verdana, sans-serif',
        'fontWeight': 'bold',
        'textAlign': 'center'
    }),
    
    # Date
    html.Div(data_row.get("Date", ""), style={
        'position': 'absolute',
        'left': '50%',
        'top': '110px',  # Positioned below the name
        'transform': 'translateX(-50%)',
        'fontSize': '28px',  # Smaller font size
        'fontFamily': 'Verdana, sans-serif',
        'textAlign': 'center'
    }),


    # Lead Leg Flexion Table
    html.Div([
        html.H4("Lead Leg Flexion", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(3, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Foot Plant", f"{data_row['Front_Leg_Footplant']:.1f}°", f"{data_row.get('Front_Leg_Footplant_percentile', ''):.1f}" if data_row.get('Front_Leg_Footplant_percentile') else ""],
            ["Max External Rotation", f"{data_row['Front_Leg_MER']:.1f}°", f"{data_row.get('Front_Leg_MER_percentile', ''):.1f}" if data_row.get('Front_Leg_MER_percentile') else ""],
            ["Release", f"{data_row['Front_Leg_Rel']:.1f}°", f"{data_row.get('Front_Leg_Rel_percentile', ''):.1f}" if data_row.get('Front_Leg_Rel_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '79px', 'top': '70px'}),

    # Trunk Position Table
    html.Div([
        html.H4("Trunk Position", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(3, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Counter Rotation at FP", f"{data_row['Trunk_Ang_Footplant']:.1f}°",
             f"{data_row.get('Trunk_Ang_Footplant_percentile', ''):.1f}" if data_row.get('Trunk_Ang_Footplant_percentile') else ""],
            ["Max External Rotation", f"{data_row['Trunk_Ang_MER']:.1f}°",
             f"{data_row.get('Trunk_Ang_MER_percentile', ''):.1f}" if data_row.get('Trunk_Ang_MER_percentile') else ""],
            ["Release", f"{data_row['Trunk_Ang_Rel']:.1f}°",
             f"{data_row.get('Trunk_Ang_Rel_percentile', ''):.1f}" if data_row.get('Trunk_Ang_Rel_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '79px', 'top': '290px'}),
    
    # Pelvis Rotation Table
    html.Div([
        html.H4("Pelvis Rotation", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(3, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Position at Foot Plant", f"{data_row['Pelvis_Ang_Footplant']:.1f}°",
             f"{data_row.get('Pelvis_Ang_Footplant_percentile', ''):.1f}" if data_row.get('Pelvis_Ang_Footplant_percentile') else ""],
            ["Max External Rotation", f"{data_row['Pelvis_Ang_MER']:.1f}°",
             f"{data_row.get('Pelvis_Ang_MER_percentile', ''):.1f}" if data_row.get('Pelvis_Ang_MER_percentile') else ""],
            ["Release", f"{data_row['Pelvis_Ang_Rel']:.1f}°",
             f"{data_row.get('Pelvis_Ang_Rel_percentile', ''):.1f}" if data_row.get('Pelvis_Ang_Rel_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '79px', 'top': '510px'}),
    
    # Hip-Shoulder Separation Table
    html.Div([
        html.H4("Hip-Shoulder Separation", style={'textAlign': 'left', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(3, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Foot Plant", f"{data_row['HSS_Footplant']:.1f}°",
             f"{data_row.get('HSS_Footplant_percentile', ''):.1f}" if data_row.get('HSS_Footplant_percentile') else ""],
            ["Max External Rotation", f"{data_row['HSS_MER']:.1f}°",
             f"{data_row.get('HSS_MER_percentile', ''):.1f}" if data_row.get('HSS_MER_percentile') else ""],
            ["Release", f"{data_row['HSS_Rel']:.1f}°",
             f"{data_row.get('HSS_Rel_percentile', ''):.1f}" if data_row.get('HSS_Rel_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '79px', 'top': '730px'}),
    
    # Shoulder ER Table
    html.Div([
        html.H4("Shoulder ER", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(2, ["", "Value", "Percentile"], [169, 69, 69], [
            ["External Rotation at FP", f"{data_row['Shld_ER_Footplant']:.1f}°",
             f"{data_row.get('Shld_ER_Footplant_percentile', ''):.1f}" if data_row.get('Shld_ER_Footplant_percentile') else ""],
            ["Max External Rotation", f"{data_row['Shld_ER_Max']:.1f}°",
             f"{data_row.get('Shld_ER_Max_percentile', ''):.1f}" if data_row.get('Shld_ER_Max_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '615px', 'top': '778px'}),
    
    # Abduction Table
    html.Div([
        html.H4("Abduction", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(2, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Horizontal Abduction", f"{data_row['Horizontal_Abduction']:.1f}°",
             f"{data_row.get('Horizontal_Abduction_percentile', ''):.1f}" if data_row.get('Horizontal_Abduction_percentile') else ""],
            ["Max External Rotation", f"{data_row['Abduction_Max']:.1f}°",
             f"{data_row.get('Abduction_Max_percentile', ''):.1f}" if data_row.get('Abduction_Max_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '1112px', 'top': '778px'}),
    
    # Kinematic Sequence Table
    html.Div([
        html.H4("Kinematic Sequence", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(4, ["", "Value", "Percentile"], [169, 69, 69], [
            ["Arm Angular Velo", f"{data_row['Arm_Ang_Velo']:.0f}°/sec",
             f"{data_row.get('Arm_Ang_Velo_percentile', ''):.1f}" if data_row.get('Arm_Ang_Velo_percentile') else ""],
            ["Hand Angular Velo", f"{data_row['Hand_Ang_Velo']:.0f}°/sec",
             f"{data_row.get('Hand_Ang_Velo_percentile', ''):.1f}" if data_row.get('Hand_Ang_Velo_percentile') else ""],
            ["Pelvis Angular Velo", f"{data_row['Pelvis_Ang_Velo']:.0f}°/sec",
             f"{data_row.get('Pelvis_Ang_Velo_percentile', ''):.1f}" if data_row.get('Pelvis_Ang_Velo_percentile') else ""],
            ["Torso Angular Velo", f"{data_row['Torso_Ang_Velo']:.0f}°/sec",
             f"{data_row.get('Torso_Ang_Velo_percentile', ''):.1f}" if data_row.get('Torso_Ang_Velo_percentile') else ""]
        ])
    ], style={'position': 'absolute', 'left': '1592px', 'top': '118px'}),
    
    # Trackman Metrics Table
    html.Div([
        html.H4("Trackman Metrics", style={'textAlign': 'center', 'marginBottom': '-13px', 'fontFamily': 'Verdana, sans-serif'}),
        create_table(8, ["", "Average", "Max"], [169, 69, 69],
                     [["Velocity", "", ""],
                      ["Spin Rate", "", ""],
                      ["Horizontal Break", "", ""],
                      ["Induced Vertical Break", "", ""],
                      ["Release Height", "", ""],
                      ["Release Side", "", ""],
                      ["VAA", "", ""],
                      ["HAA", "", ""]])
    ], style={'position': 'absolute', 'left': '1592px', 'top': '468px'}),
    
    # Score Table
    html.Div([
        html.Table([
            html.Tbody([
                html.Tr([
                    html.Td("Score", style={
                        'fontWeight': 'bold', 'fontSize': '24px', 'fontFamily': 'Verdana, sans-serif',
                        'textAlign': 'center', 'width': '169px', 'height': '36px', 'padding': '4px'
                    }),
                    html.Td(f"{data_row['Score']:.1f}" if 'Score' in data_row else "N/A", style={
                        'fontWeight': 'bold', 'fontSize': '24px', 'fontFamily': 'Verdana, sans-serif',
                        'textAlign': 'center', 'width': '69px', 'height': '36px', 'padding': '4px'
                    })
                ]),
                html.Tr([
                    html.Td("Percentile", style={
                        'textAlign': 'center', 'fontFamily': 'Verdana, sans-serif',
                        'width': '169px', 'height': '36px', 'padding': '4px'
                    }),
                    html.Td(
                        f"{data_row.get('Score_percentile', ''):.1f}" if data_row.get('Score_percentile') is not None else "",
                        style={
                            'textAlign': 'center', 'fontFamily': 'Verdana, sans-serif',
                            'width': '69px', 'height': '36px', 'padding': '4px'
                        }
                    )
                ])
            ])
        ])
    ], style={'position': 'absolute', 'left': '893px', 'top': '628px'}),

])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8052)


KeyError: 'Front_Leg_Footplant'

In [ ]:
import pandas as pd
import sqlite3
import json

# File paths
db_path = "Cover_Page.db"  # Path to your SQLite database
excel_path = "Metric Outputs3.xlsx"  # Path to your Excel mapping file
output_json_path = "output_payload.json"  # Path to save the output JSON

# Load the Excel file
mapping_df = pd.read_excel(excel_path)

# Extract the list of required columns from the database (Column A in Excel)
required_columns = mapping_df["originColumn"].dropna().tolist()

# Ensure "Score" is in the required columns to retrieve it for filtering
if "Score" not in required_columns:
    required_columns.append("Score")

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Get valid column names from the database
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(variables);")
valid_columns = {col[1] for col in cursor.fetchall()}

# Filter out invalid or problematic column names
filtered_columns = [col for col in required_columns if col in valid_columns]

# Query the database for the filtered columns
query = f"SELECT {', '.join(filtered_columns)} FROM variables"
db_data = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Identify the row with the highest Score
if "Score" in db_data.columns:
    max_score_row = db_data.loc[db_data["Score"].idxmax()]
else:
    raise ValueError("The 'Score' column is not present in the database or the mapping file.")

# Initialize the JSON payload
payload = {
    "athleteUuid": "",
    "level": "PRO",
    "score": max_score_row["Score"],  # Use the score from the row with the highest score
    "metrics": []
}

# Iterate through the mapping file and populate metrics for the row with the highest score
for _, map_row in mapping_df.iterrows():
    column = map_row["originColumn"]
    if column in max_score_row:
        metric_entry = {
            "category": map_row["category"],
            "name": map_row["name"],
            "value": max_score_row[column],
            "valueUnit": map_row["valueUnit"],
            "orientation": map_row["orientation"] if pd.notna(map_row["orientation"]) else None,
        }
        payload["metrics"].append(metric_entry)

# Save the JSON payload to a file
with open(output_json_path, "w") as json_file:
    json.dump(payload, json_file, indent=4)

print(f"JSON payload saved to {output_json_path}")


In [ ]:
# Creates JSON files based on schema for entire database (HS, College, Pro) 
# 
# import pandas as pd
# import sqlite3
# import json
# 
# # File paths
# metrics_mapping_path = "Metric Outputs3.xlsx"  # Path to your Excel mapping file
# grading_db_path = "grading_equation_reference_data_HS.db"  # Path to the SQLite database
# output_json_path = "athlete_reports.json"  # Path to save the output JSON
# 
# # Load the Excel mapping file
# mapping_df = pd.read_excel(metrics_mapping_path)
# 
# # Extract the list of required columns from the mapping (Column A in Excel)
# required_columns = mapping_df["originColumn"].dropna().tolist()
# 
# # Connect to the SQLite database
# conn = sqlite3.connect(grading_db_path)
# 
# # Fetch valid column names from the database
# cursor = conn.cursor()
# cursor.execute("PRAGMA table_info(variables);")
# valid_columns = {col[1] for col in cursor.fetchall()}  # Column names from the database
# 
# # Filter required columns to include only valid column names
# filtered_columns = [col for col in required_columns if col in valid_columns]
# 
# # Query the database with valid column names
# query = f"SELECT {', '.join(filtered_columns)} FROM variables"
# data = pd.read_sql_query(query, conn)
# 
# # Close the database connection
# conn.close()
# 
# # Replace NaN values with 0
# data = data.fillna(0)
# 
# # Initialize the JSON payload
# payload = {
#     "level": "PRO",
#     "athleteReports": []
# }
# 
# # Generate the athlete reports
# for _, row in data.iterrows():
#     athlete_report = {
#         "score": row["Score"],  # Ensure "Score" is included in the queried columns
#         "metrics": []
#     }
#     
#     for _, map_row in mapping_df.iterrows():
#         origin_column = map_row["originColumn"]
#         if origin_column in row:
#             metric_entry = {
#                 "category": map_row["category"],
#                 "name": map_row["name"],
#                 "value": row[origin_column],
#                 "valueUnit": map_row["valueUnit"],
#                 "orientation": map_row["orientation"] if pd.notna(map_row["orientation"]) else None,
#             }
#             athlete_report["metrics"].append(metric_entry)
#     
#     payload["athleteReports"].append(athlete_report)
# 
# # Save the JSON payload to a file
# with open(output_json_path, "w") as json_file:
#     json.dump(payload, json_file, indent=4)
# 
# print(f"JSON payload saved to {output_json_path}")
